In [17]:
# # Nur bei der Ausführung des Codes in Google Colab verwenden!
# !git clone https://github.com/edadunashvili/KaFk.git

In [18]:
# # Nur bei der Ausführung des Codes in Google Colab verwenden!
# import os
# try:
#     os.chdir('/content/KaFk')
#     print("Verzeichnis erfolgreich gewechselt!")
# except FileNotFoundError:
#     print("Verzeichnis nicht gefunden.")
# except Exception as e:
#     print(f"Ein Fehler ist aufgetreten: {e}")

In [19]:
# # Nur bei der Ausführung des Codes in Google Colab verwenden!
# from IPython.display import HTML, display
# def set_css():
#   display(HTML('''
#   <style>
#     pre {
#         white-space: pre-wrap;
#     }
#   </style>
#   '''))
# get_ipython().events.register('pre_run_cell', set_css)

# Inhaltsannotation(en)

### README

###### Tool zur Auswertung der annotierten Texte  in Form von Motivlisten

##### Anpassbare Variablen:
if ("tg" in corp.attrib.get("n", "") or "xtk" in corp.attrib.get("n", "")):Sicherstellen, dass `n` den gesuchte Textkorpus z.B 'xtk', darf nicht leergelassen sein

    
if "" in tn: Sicherstellen, dass `tn` das gesuchte Typ z.B 'a300' enthält, falls leergelassen greift die Suche das gesamte Repertoire zu.
    

In [20]:
import xml.etree.ElementTree as ET

# XML-Datei laden und Wurzel abrufen
root_node = ET.parse('gesamt_märchen.xml').getroot()

# Namespace definieren
NS = {'tei': 'http://www.tei-c.org/ns/1.0'}
filter_ident = ""  # Kann auf "" gesetzt werden, um keinen Filter anzuwenden
unique_schema_set = set()

# Durchlaufen des XML-Baums
for corp in root_node.findall(".//tei:teiCorpus", NS):
    if ("tg" in corp.attrib.get("n", "") or "xtk" in corp.attrib.get("n", "")):
        for ganze in corp.findall(".//tei:TEI", NS):
            language = ganze.find(".//tei:teiHeader//tei:langUsage/tei:language", NS)
            # Bedingung für Sprache: anwenden, wenn filter_ident definiert ist, oder überspringen
            if language is not None and (not filter_ident or language.attrib.get('ident') == filter_ident):
                for text in ganze.findall(".//tei:text", NS):
                    if "" in text.attrib['{http://www.w3.org/XML/1998/namespace}id']:  
                        for segm in text.findall(".//tei:seg", NS):
                            tn = segm.attrib.get('{www.dglab.uni-jena.de/vmf/a}ana', '')
                            if "a1164" in tn:  # Sicherstellen, dass `tn` das gesuchte Muster enthält
                                chisch = text.attrib.get('{http://www.w3.org/XML/1998/namespace}id', '')
                                if chisch:
                                    unique_schema_set.add(chisch)

# Ergebnisse in Datei schreiben
with open("TextClass.txt", 'w', encoding='utf-8') as f:
    f.write(str(sorted(unique_schema_set)))


In [21]:
import ast
import xml.etree.ElementTree as ET

def setze_variable_aus_datei(dateiname: str, variablenname: str):
    try:
        with open(dateiname, "r", encoding="utf-8") as datei:
            inhalt = datei.read().strip()
        wert = ast.literal_eval(inhalt)
        globals()[variablenname] = wert
        return wert
    except Exception as e:
        print(f"Fehler beim Einlesen und Setzen der Variable: {e}")
        return None

# 1. Mehrere `chisch`-Werte definieren
chisch = setze_variable_aus_datei("TextClass.txt", "chisch")

# Überprüfen, ob die Variable korrekt gesetzt wurde
print(chisch)


['zyx_cr_agx_rus_3ß', 'zyx_cr_ava_rus_115ß', 'zyx_cr_kum_rus_28ß', 'zyx_cr_lez_rus_11ß', 'zyx_cr_nog_rus_46ß']


In [22]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

# Modul zur Speicherung in einer zentralen Textdatei
def speichere_gesamt_ergebnisse(dateiname, daten):
    """
    Speichert die Gesamtergebnisse in eine Textdatei.
    
    :param dateiname: Name der Textdatei
    :param daten: Ergebnisse in Form eines Dictionaries {chisch: [Daten]}
    """
    with open(dateiname, 'w', encoding='utf-8') as f:
        for ch, ergebnisse in daten.items():
            f.write(f"\n{ch}:\n")
            for zeile in ergebnisse:
                f.write(f"{zeile}\n")

# Mehrere `chisch`-Werte definieren
chisch = setze_variable_aus_datei("TextClass.txt", "chisch")
# chisch = ['zyx_cr_lbe_rus_43'] # 1.1. Alternativ: Einen chisch-Wert eingeben

root_node = ET.parse('gesamt_märchen.xml').getroot()

# Speichern der Märchendaten für jedes `chisch`
maerchen_daten = {ch: "" for ch in chisch}

# 2. Schleife für jeden `chisch`-Wert in der Liste
for ch in chisch:
    for ganze in root_node.findall(".//{http://www.tei-c.org/ns/1.0}text"):
        if ganze.attrib.get('{http://www.w3.org/XML/1998/namespace}id') == ch:
            for body in ganze.findall(".//{http://www.tei-c.org/ns/1.0}body"):
                for absatz in body.findall(".//{http://www.tei-c.org/ns/1.0}p"):
                    for phrase in absatz.findall(".//{http://www.tei-c.org/ns/1.0}seg"):
                        try:
                            # Extrahieren der relevanten Attribute
                            labela = phrase.attrib['{www.dglab.uni-jena.de/vmf/a}ana']
                            labelb1 = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/b1}ana', 'N')
                            labelb2 = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/b2}ana', 'N')
                            labelb3 = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/b3}ana', 'N')
                            labelb4 = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/b4}ana', 'N')
                            labelb5 = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/b5}ana', 'N')
                            labelc1 = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/c1}ana', 'N')
                            labelc2 = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/c2}ana', 'N')
                            labelc3 = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/c3}ana', 'N')
                            labelc4 = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/c4}ana', 'N')
                            labelc5 = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/c5}ana', 'N')
                            labeld = phrase.attrib.get('{www.dglab.uni-jena.de/vmf/d}ana', 'N')
                            quelle = ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']  
                            inhalt = (phrase.text.lower().strip()
                                      .replace('|','').replace(':','').replace("ä","ae").replace("ü","ue")
                                      .replace("ö","oe").replace("ß","ss").replace(",","" ).replace("«","")
                                      .replace("»","" ).replace(".","" ).replace(";","" ).replace('"',"")
                                      .replace("?","" ).replace("!","" ).replace("-"," " ).replace("'", "")
                                      .replace("(", "" ).replace(")", "" ).replace("\n","" ).replace("  ", " "))
                            if labela.startswith('a'):
                                # Speichern der Märchendaten in der entsprechenden `chisch`-Liste
                                maerchen_daten[ch] += f"{quelle},{labela},{labelb1},{labelc1},{labelb2},{labelc2},{labelb3},{labelc3},{labelb4},{labelc4},{labelb5},{labelc5},{labeld},{inhalt},0\n"
                        except KeyError as e:
                            print(f"Fehlendes Attribut: {e}")

# 3. Ausgabestruktur für jeden `chisch`-Wert erstellen
gesamt_ergebnisse = {}

# Erstellen der CSV-Dateien und Verarbeiten der Daten
unique_values_per_ch = {}
result_data_per_ch = {}
for ch in chisch:
    # CSV-Datei erstellen
    with open(f"text_{ch}.csv", 'w', encoding='utf-8') as f:
        f.write('quelle,labela,labelb1,labelc1,labelb2,labelc2,labelb3,labelc3,labelb4,labelc4,labelb5,labelc5,labeld,inhalt,index_binar\n')
        f.write(maerchen_daten[ch])

    # Index-Strings erstellen
    abcd_data = []
    df = pd.read_csv(f"text_{ch}.csv", encoding='utf-8')

    for index, row in df.iterrows():
        quelle = row['quelle']
        labela = row['labela']
        for i in range(1, 6):
            labelb = row[f'labelb{i}']
            labelc = row[f'labelc{i}']
            labeld = row['labeld']
            inhalt = row['inhalt']
            abcd_data.append(f"{labela}:{labelb}:{labelc}:{labeld}")

    # Filterung der Daten
    filtered_data = [line for line in abcd_data if ':N:' not in line]

    # Extrahieren einzigartiger Werte vor dem ersten ':'
    unique_values = set(line.split(':')[0] for line in filtered_data)
    unique_values_per_ch[ch] = unique_values
    result_data_per_ch[ch] = filtered_data

    # Speichern der gefilterten Daten in der Gesamt-Ausgabeliste
    gesamt_ergebnisse[ch] = filtered_data

# Zählen der `chisch`-Werte mit und ohne Ergebnisse
chisch_mit_ergebnissen = [ch for ch, ergebnisse in gesamt_ergebnisse.items() if ergebnisse]
chisch_ohne_ergebnisse = [ch for ch, ergebnisse in gesamt_ergebnisse.items() if not ergebnisse]

print(f"Anzahl der chisch-Werte mit Ergebnissen: {len(chisch_mit_ergebnissen)}")
print(f"Anzahl der chisch-Werte ohne Ergebnisse: {len(chisch_ohne_ergebnisse)}")

# Parsen der `fk/vmf_a.xsd`-Datei und Überprüfen von Werten mit Leerzeichen
fk_tree = ET.parse('kf/vmf_a.xsd').getroot()
enum_elements = fk_tree.findall(".//{http://www.w3.org/2001/XMLSchema}enumeration")
enum_values_with_spaces = {enum.attrib['value'] for enum in enum_elements if ' ' in enum.attrib['value']}

# Ausgabe der Ergebnisse
for ch, unique_values in unique_values_per_ch.items():
    print(f"\n\n{ch}")
    print("-----------------")
    matching_values = [
    value for value in enum_values_with_spaces 
    if any(value.split(' ')[0] == uval for uval in unique_values)
]
    print(*matching_values, sep='\n')
    print("-----------------")

    # Ergebnisse mit Werten anzeigen
    if ch in result_data_per_ch:
        #print("Gefilterte Ergebnisse:")
        for result in result_data_per_ch[ch]:
            print(result)

# Speichern in einer Textdatei
speichere_gesamt_ergebnisse("TextClassaLaufende.txt", gesamt_ergebnisse)

# Löschen der erstellten CSV-Dateien
for ch in chisch:
    try:
        os.remove(f"text_{ch}.csv")  # Löschen der CSV-Datei
    except FileNotFoundError:
        print(f"Datei text_{ch}.csv wurde nicht gefunden und konnte nicht gelöscht werden.")
    except Exception as e:
        print(f"Fehler beim Löschen der Datei text_{ch}.csv: {e}")



Anzahl der chisch-Werte mit Ergebnissen: 5
Anzahl der chisch-Werte ohne Ergebnisse: 0


zyx_cr_agx_rus_3ß
-----------------
a1164 Belfagor
a1862B The Sham Physician and the Devil in Partnership
-----------------
a1164:h:Unerträgliche_Ungeheuerlichkeit:rHD_fHD
a1164:F:Ungewünschtes_Wesen_beseitigen:rHD_fHD
a1164:H:MISSETAT_wie_Jemanden_im_Stich_lassen:rHD_fHD
a1862B:Hh:Sorge_um_Hilfsbedürftige:rHD_fAN
a1862B:F:VERTRAG_abschließen_oder_verkünden:rHD_fAN
a1862B:HF:Eine_Bedrohung_VORTÄUSCHEN:rHD_fAN
a1862B:HF:Heldentat_VORTÄUSCHEN:rHD_fAN
a1862B:F:WASSERMANGEL_beheben:rHD_rHP
a1862B:F:Menschliche_OPFERGABE_beheben:rHD_rHP
a1862B:F:VERTRAG_abschließen_oder_verkünden:rHD_rHP
a1862B:F:Um_einen_Auftrag_werben:rHD_rHP
a1862B:F:VERTRAG_abschließen_oder_verkünden:rHD_rHP
a1862B:H:Heldentat_VORTÄUSCHEN:rHD_fAN
a1862B:H:Dem_VORTÄUSCHEN_nachgehen:rHD_fAN
a1862B:H:VERTREIBUNG:rHD_fAN
a1862B:H:WASSERMANGEL_beheben:rHD_fAN
a1862B:H:Menschliche_OPFERGABE_beheben:rHD_fAN
a1862B:H:VERTRAGsbedingungen_erfü